Preprocesamiento
- No hace falta estandarizar, no estaría tampoco mal
- Agrupar las binarias (has..) en grupos
  - has_secondary_use (hay que añadir más para las combinaciones)
  - has_superstructure
- geo_level3 o 2 quizás se puedan agrupar en intervalos
- Pasar las categóricas a números ? En sí las letras no tienen significado
- Quizás agrupar la edad en intervalos
- Ver qué hacer con las duplicadas
- Los árboles son robustos al ruido, así que las variables extremadamente 
desbalanceadas no deberían hacer mucho daño. Considerar si quitar o no según el
coste computacional que haya en principio.
- Age de 995 mantenerlo, a ver si el árbol consigue algo del estilo if age>200 ...

- Quizá la distancia, al ser datos del mismo terremoto sea muy relevante. 
Se podría reflejar en una combinación de las has_structure y geom_ids 
(el epicentro estaría por unos valores concretos de geom_id)

In [27]:
################################################################################
# Libraries
################################################################################

import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [28]:
################################################################################
# Load data
################################################################################

train_values = pd.read_csv("data/train_values.csv")
test_values  = pd.read_csv("data/test_values.csv")
train_labels = pd.read_csv("data/train_labels.csv")

preprocessed_train = train_values
preprocessed_test = test_values


In [29]:
train_values["geo_level_1_id"]

0          6
1          8
2         21
3         22
4         11
          ..
260596    25
260597    17
260598    17
260599    26
260600    21
Name: geo_level_1_id, Length: 260601, dtype: int64

In [30]:
preprocessed_train

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,...,0,0,0,0,0,0,0,0,0,0
260597,669485,17,715,2060,2,0,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
260598,602512,17,51,8163,3,55,6,7,t,r,...,0,0,0,0,0,0,0,0,0,0
260599,151409,26,39,1851,2,10,14,6,t,r,...,0,0,0,0,0,0,0,0,0,0


In [31]:
for i,c in enumerate(preprocessed_train.columns):
    print(i, c)

0 building_id
1 geo_level_1_id
2 geo_level_2_id
3 geo_level_3_id
4 count_floors_pre_eq
5 age
6 area_percentage
7 height_percentage
8 land_surface_condition
9 foundation_type
10 roof_type
11 ground_floor_type
12 other_floor_type
13 position
14 plan_configuration
15 has_superstructure_adobe_mud
16 has_superstructure_mud_mortar_stone
17 has_superstructure_stone_flag
18 has_superstructure_cement_mortar_stone
19 has_superstructure_mud_mortar_brick
20 has_superstructure_cement_mortar_brick
21 has_superstructure_timber
22 has_superstructure_bamboo
23 has_superstructure_rc_non_engineered
24 has_superstructure_rc_engineered
25 has_superstructure_other
26 legal_ownership_status
27 count_families
28 has_secondary_use
29 has_secondary_use_agriculture
30 has_secondary_use_hotel
31 has_secondary_use_rental
32 has_secondary_use_institution
33 has_secondary_use_school
34 has_secondary_use_industry
35 has_secondary_use_health_post
36 has_secondary_use_gov_office
37 has_secondary_use_use_police
38 has_s

Join columns
- 15...25 (has superstructure) (they are not dummy variables, use combination)
- 29...38 (has sencondary use) (probably dummy variables)

Remove 28, not needed

Reorder numerics next to numerics

In [32]:
preprocessed_train[["has_superstructure_adobe_mud", "has_superstructure_stone_flag"]]

,has_superstructure_adobe_mud,has_superstructure_stone_flag
0,1,0
1,0,0
2,0,0
3,0,0
4,1,0
...,...,...
260596,0,0
260597,0,0
260598,0,0
260599,0,0


In [33]:
# Changing column names
# df = preprocessed_train.rename(columns={x:y for x,y in zip(preprocessed_train.columns,range(0,len(preprocessed_train.columns)))})


In [34]:
# Join has_superstructure (binary) columns has strings
# Get selected rows to string
df = preprocessed_train.iloc[:,15:25].astype(str)
df_test = preprocessed_test.iloc[:,15:25].astype(str)

# Join them
preprocessed_train["has_superstructure"] = df.apply(lambda x: ''.join(x), axis=1)
preprocessed_test["has_superstructure"] = df_test.apply(lambda x: ''.join(x), axis=1)

In [35]:
# Join has_secondary (binary) columns has strings
# Get selected rows to string
df = preprocessed_train.iloc[:,29:38].astype(str)
df_test = preprocessed_test.iloc[:,29:38].astype(str)

# Join them
preprocessed_train["has_secondary"] = df.apply(lambda x: ''.join(x), axis=1)
preprocessed_test["has_secondary"] = df_test.apply(lambda x: ''.join(x), axis=1)

In [36]:
# Remove joined columns
removed_cols = list(range(15,26)) + list(range(29,39))
preprocessed_train = preprocessed_train.drop(columns=preprocessed_train.columns[removed_cols])
preprocessed_test = preprocessed_test.drop(columns=preprocessed_test.columns[removed_cols])

preprocessed_train

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,legal_ownership_status,count_families,has_secondary_use,has_superstructure,has_secondary
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,v,1,0,1100000000,000000000
1,28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,v,1,0,0100000000,000000000
2,94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,v,1,0,0100000000,000000000
3,590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,v,1,0,0100001100,000000000
4,201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,v,1,0,1000000000,000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,n,f,j,s,q,v,1,0,0100000000,000000000
260597,669485,17,715,2060,2,0,6,5,t,r,n,f,q,s,d,v,1,0,0100000000,000000000
260598,602512,17,51,8163,3,55,6,7,t,r,q,f,q,s,d,v,1,0,0100000000,000000000
260599,151409,26,39,1851,2,10,14,6,t,r,x,v,s,j,d,v,1,0,0000010000,000000000


In [37]:
# Convert new cols to int
# preprocessed_train["has_superstructure"] = preprocessed_train["has_superstructure"].apply(lambda x: int(x,2))
# preprocessed_train["has_secondary"] = preprocessed_train["has_secondary"].apply(lambda x: int(x,2))
# preprocessed_train

In [38]:
# Convert to categorical in order (TODO: KEEP ORIGINAL MAPPING)
# MEJOR: Leerlo como factor en R

# preprocessed_train.has_secondary = preprocessed_train.has_secondary.astype("category").cat.codes
# preprocessed_train.has_superstructure = preprocessed_train.has_superstructure.astype('category').cat.codes

# preprocessed_train

In [39]:
# No hay duplicados en los ids, se pueden quitar
# preprocessed_train.building_id.duplicated

preprocessed_train = preprocessed_train.drop(columns="building_id")
preprocessed_test = preprocessed_test.drop(columns="building_id")
preprocessed_train


,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,legal_ownership_status,count_families,has_secondary_use,has_superstructure,has_secondary
0,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,v,1,0,1100000000,000000000
1,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,v,1,0,0100000000,000000000
2,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,v,1,0,0100000000,000000000
3,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,v,1,0,0100001100,000000000
4,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,v,1,0,1000000000,000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,25,1335,1621,1,55,6,3,n,r,n,f,j,s,q,v,1,0,0100000000,000000000
260597,17,715,2060,2,0,6,5,t,r,n,f,q,s,d,v,1,0,0100000000,000000000
260598,17,51,8163,3,55,6,7,t,r,q,f,q,s,d,v,1,0,0100000000,000000000
260599,26,39,1851,2,10,14,6,t,r,x,v,s,j,d,v,1,0,0000010000,000000000


In [40]:
# has_secondary_use is enconded in has_secondary, remove it
preprocessed_train = preprocessed_train.drop(columns="has_secondary_use")
preprocessed_test = preprocessed_test.drop(columns="has_secondary_use")
preprocessed_train

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,legal_ownership_status,count_families,has_superstructure,has_secondary
0,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,v,1,1100000000,000000000
1,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,v,1,0100000000,000000000
2,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,v,1,0100000000,000000000
3,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,v,1,0100001100,000000000
4,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,v,1,1000000000,000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,25,1335,1621,1,55,6,3,n,r,n,f,j,s,q,v,1,0100000000,000000000
260597,17,715,2060,2,0,6,5,t,r,n,f,q,s,d,v,1,0100000000,000000000
260598,17,51,8163,3,55,6,7,t,r,q,f,q,s,d,v,1,0100000000,000000000
260599,26,39,1851,2,10,14,6,t,r,x,v,s,j,d,v,1,0000010000,000000000


In [41]:
############################################################################
# Preprocesado Nº 2
############################################################################

# Quitamos geo_2 y geo_3. Tienen demasiadas categorías diferentes para los árboles,
# y no se solapan
preprocessed_train = preprocessed_train.drop(columns=["geo_level_2_id","geo_level_3_id"])
preprocessed_test = preprocessed_test.drop(columns=["geo_level_2_id","geo_level_3_id"])


# En estas dos predomina (+85%) una clase y el resto no sirve para determinar
# ninguna etiqueta (se mantiene la proporción o las 3 están representadas)
preprocessed_train = preprocessed_train.drop(columns=["plan_configuration","legal_ownership_status"])
preprocessed_test = preprocessed_test.drop(columns=["plan_configuration","legal_ownership_status"])

In [47]:
# Altura y nº de plantas altamente correladas, categorizamos nº de plantas y quitamos altura
# Solo una instancia con 9 plantas, y las anteriores no siguen el mismo patrón (que todas sufrieran el mismo tipo de daño)
# Como tenemos proporciones muy similares, juntamos las +5 con ella
preprocessed_train["count_floors_pre_eq"] = preprocessed_train["count_floors_pre_eq"].replace({6:5, 7:5, 8:5, 9:5})
preprocessed_test["count_floors_pre_eq"] = preprocessed_test["count_floors_pre_eq"].replace({6:5, 7:5, 8:5, 9:5})

# No nos interesan variables numéricas en los árboles, acabarían discretizándose
preprocessed_train = preprocessed_train.drop(columns=["height_percentage"])
preprocessed_test = preprocessed_test.drop(columns=["height_percentage"])


KeyError: "['height_percentage'] not found in axis"

In [48]:
pd.unique(preprocessed_train.count_floors_pre_eq)

array([2, 3, 1, 4, 5], dtype=int64)

In [44]:
################################################################################
# Write data
################################################################################
preprocessed_train.to_csv("data/preprocessed_train.csv", index=False)
preprocessed_test.to_csv("data/preprocessed_test.csv", index=False)